In [39]:
# Given prior probabilities and test result likelihoods
prior1 = 0.001  # p(X = 1)
FN = 0.01  # p(Y = 0 | X = 1)
FP = 0.1  # p(Y = 1 | X = 0)

# Calculate the complementary probabilities
prior0 = 1 - prior1 # p(X = 0)
TP = 1 - FN # p(Y = 1 | X = 1)
TN = 1 - FP  # p(Y = 0 | X = 0)

## Q1

In [40]:
# 1(a) calculate p(X = 1 | Y = 1) using Bayes Rule
normalization_factor_1a = prior0 * FP + prior1 * TP # p(Y = 1)
posterior_1a = TP * prior1 / normalization_factor_1a
posterior_1a

0.009812667261373772

In [41]:
# 1(b) calculate p(X = 1 | Y = 0) using Bayes Rule
normalization_factor_1b = prior0 * TN + prior1 * FN # p(Y = 0)
posterior_1b = FN * prior1 / normalization_factor_1b
posterior_1b 

1.112210964175685e-05

## Q3

In [42]:
# Given prior probabilities and test result likelihoods
x1 = 0.001  # p(X = 1)
y0_x1 = 0.01  # p(Y = 0 | X = 1)
y1_x0 = 0.1  # p(Y = 1 | X = 0)

# Calculate the complementary probabilities
x0 = 1 - x1 # p(X = 0)
y1_x1 = 1 - FN # p(Y = 1 | X = 1)
y0_x0 = 1 - FP  # p(Y = 0 | X = 0)

x1_y1 = posterior_1a
x0_y1 = 1 - x1_y1 # p(X = 0 | Y = 1)

x1_y0 = posterior_1b
x0_y0 = 1 - x1_y0 # p(X = 0 | Y = 0)

In [43]:
# 3(a) calculate p(X = 1 | Y = 0) using Bayesian estimator
likelihood = y1_x1
prior = x1_y1
evidence = y1_x0 * (1-prior) + y1_x1 * prior
posterior = likelihood * prior / evidence
posterior

0.08934285011075559

In [44]:
#3(b)
def recursive_update_posterior(posterior, TP, FP, count=0):
    """
    Recursive function to update the posterior probability based on new positive test results.
    
    :param posterior: The current posterior probability of having the disease.
    :param true_positive: The true positive rate of the test.
    :param false_positive: The false positive rate of the test.
    :return: The updated posterior probability.
    """
    # calculate evidence
    evidence = TP * posterior + FP * (1 - posterior)
    # update the posterior probability
    updated_posterior = (TP * posterior) / evidence
     
    # num of iter
    count += 1
    
    # updated posterior is greater than 0.5, return the value and number of tests
    if updated_posterior > 0.5:
        return updated_posterior, count
    else:
        # call the function recursively
        return recursive_update_posterior(updated_posterior, TP, FP, count)

# initial posterior
posterior_i = x1_y1
posterior_final, count = recursive_update_posterior(posterior_i, y1_x1, y1_x0, count=0)
posterior_final, count

(0.9057987969233379, 3)

## Q4

In [45]:
from sympy import symbols, Matrix
sensitivity = 0.95  # Probability of testing positive given infected
specificity = 0.95  # Probability of testing negative given healthy

# Initial belief 
pi_0 = Matrix([1, 0])

T = Matrix([
    [0.999, 0.05],
    [0.001, 0.95]
])

def bayesian_update_one_day(pi_previous, test_result, T):
    M_yt = Matrix([[specificity, 0], [0, sensitivity]]) if test_result else Matrix([[1 - sensitivity, 0], [0, 1 - specificity]])

    unnormalized_pi = M_yt * T.transpose() * pi_previous

    # Normalize the belief state
    Z = sum(unnormalized_pi)
    normalized_pi = unnormalized_pi / Z

    return normalized_pi

# Iterate through 98 negative tests
pi = pi_0
for day in range(1, 99):
    pi = bayesian_update_one_day(pi, False, T)

# 99th day with a positive test
pi_99 = bayesian_update_one_day(pi, True, T)

# probability of being infected on day 99
probability_99 = pi_99[1]
probability_99


0.498564207209785